# This Notebook is created for scraping green-acres.fr .

### Import the modules

In [1]:
import pandas as pd
import numpy as np
import requests, bs4
from scrapy.selector import Selector
from scrapy.http import HtmlResponse
from bs4 import BeautifulSoup
import time
import random
import re
pd.set_option('display.float_format', lambda x: '%.5f' % x)

In [2]:
agents = [
'Mozilla/5.0 (Windows NT 6.1) AppleWebKit/537.36 (KHTML, like Gecko)',
'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_1) AppleWebKit/537.36 (KHTML, like Gecko)',
'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko)',
'Mozilla/5.0 (Windows NT 6.3; WOW64) AppleWebKit/537.36 (KHTML, like Gecko)',
'Mozilla/5.0 (Windows NT 6.4; WOW64) AppleWebKit/537.36 (KHTML, like Gecko)',
'Mozilla/5.0 (Windows NT 5.1) AppleWebKit/537.36 (KHTML, like Gecko)',
'Mozilla/5.0 (Windows NT 6.1; WOW64; Trident/7.0; AS; rv:11.0) like Gecko',
'Mozilla/5.0 (compatible, MSIE 11, Windows NT 6.3; Trident/7.0; rv:11.0) like Gecko',
'Mozilla/5.0 (compatible; MSIE 10.0; Windows NT 6.1; WOW64; Trident/6.0)',
'Mozilla/5.0 (compatible; MSIE 10.0; Windows NT 6.1; Trident/6.0)']

In [3]:
number_of_pages_link = "https://www.green-acres.fr/en/prog_show_properties-order-date_d-lg-en-cn-fr-i-24-city_id-dp_06.html?p_n=1"
page_number_xpath = "substring-before(substring-after(normalize-space(//p[@class='pagination-info']),'of'),' p')"
try:
    headers = {"User-Agent":random.choice(agents)}
    response = requests.get(number_of_pages_link,headers=headers,proxies=None)
    total_page_number = Selector(text=response.text).xpath(page_number_xpath).extract()[0]
except:
    print("not able to scrap page number")

In [4]:
total_page_number = int(int(total_page_number.replace(' ','').replace(',',''))/24)+1
print(total_page_number)

448


In [8]:
half_total_page_number = int(total_page_number/2)
print(half_total_page_number)

224


### Scraping for all house announcements' urls.

In [9]:
start = time.time()
pagination_link = "https://www.green-acres.fr/en/prog_show_properties-order-date_d-lg-en-cn-fr-i-24-city_id-dp_06.html?p_n="
mylist = []
# Range Number checked before scraping. 
for page_number in range(1,half_total_page_number):
    try:
        headers = {"User-Agent":random.choice(agents)}
        if page_number%25==0:
            print("Finisihed 25 page")
        url = pagination_link + str(page_number)
       # Getting the webpage, creating a Response object.
        response = requests.get(url,headers=headers,proxies=None)

        # Extracting the source code of the page.
        data = response.text

        # Passing the source code to BeautifulSoup to create a BeautifulSoup object for it.
        ## Not so much necessary, just looking the response.text as a shape of website.
        soup = BeautifulSoup(data, 'lxml')

        # Extracting all the <a> tags into a list.
        tags = soup.find_all('a')

        # Extracting URLs from the attribute href in the <a> tags.
        for tag in tags:
            unit = str(tag.get('href'))
            if "/properties/" in unit:
                unit = "https://www.green-acres.fr" + unit
                mylist.append(unit)
        # For soft requesting.
        time.sleep(1)
        
    except Exception as e:
        print("Not able to connect",url,"Error: ",e)

end = time.time()
print(end - start," seconds execution time")

Finisihed 25 page
Finisihed 25 page
Finisihed 25 page
Finisihed 25 page
Finisihed 25 page
Finisihed 25 page
Finisihed 25 page
Finisihed 25 page
653.6754720211029  seconds execution time


In [10]:
# Dropping duplicate elements if exist.
mylist = list(set(mylist))
print("inside mylist ",len(mylist)," number of links appended")

inside mylist  5352  number of links appended


In [11]:
title_xpath = "normalize-space(//h1[@class='item-title'])"

price_xpath = "normalize-space(//span[@class='price'])"
price_per_sqm_xpath = "substring-after(normalize-space(//p[@class='text price-text'][2]),'m² ')"
sqm_area_xpath = "substring-before(normalize-space(//div[@class='item-content-part main-characteristics']//li[1]//p[1]),' ')"
house_summary_xpath = "normalize-space(//div[@class='item-content-part']//p[@class='text'])"
reference_no_xpath = "normalize-space(//p[@class='text price-text'][4])"
region_name_xpath = "normalize-space(//div[@class='breadcrumbs-area']//li[2])"
departmant_name_xpath = "normalize-space(//div[@class='breadcrumbs-area']//li[3])"
city_name_xpath = "normalize-space(//div[@class='breadcrumbs-area']//li[4])"

## Need to do regex.
house_properties_xpath = "normalize-space(//div[@class='item-content-part main-characteristics'])"

popularity_xpath = "normalize-space(//div[@class='col-sm-12 col-md-6'])"

In [12]:
start = time.time()

title_list = []
price_value_list = []
price_currency_list = []

price_per_sqm_list = []
sqm_area_list = []

reference_no_list = []
house_summary_list = []

region_name_list = []
departmant_name_list = []
city_name_list = []

latitude_list = []
longitude_list = []

house_url_list = []

house_properties_list = []
view_number_list = []
publish_date_list = []

bedrooms_list = []
shower_rooms_list = []
bathrooms_list = []
rooms_list = []
sqm_list = []
ges_list = []
dpe_list = []


start = time.time()
counter = 0
for house_url in mylist:
    headers = {"User-Agent":random.choice(agents)}
    try:
        page = requests.get(house_url,headers=headers,proxies=None)
    except:
        print("there is no url like:",house_url)
        
    try:
        title = Selector(text=page.text).xpath(title_xpath).extract()[0]
        title_list.append(title)
        #print(title)
    except:
        title = np.nan
        title_list.append(title)
    
    try:
        price = Selector(text=page.text).xpath(price_xpath).extract()[0]
        price_currency = price.split(' ')[0]
        price_value = price.split(' ')[1]
        price_value = int(price_value.replace(',',''))
        #print(price_value)
        #print(price_currency)
        price_value_list.append(price_value)
        price_currency_list.append(price_currency)
    except:
        price_value = np.nan
        price_currency = np.nan
        price_value_list.append(price_value)
        price_currency_list.append(price_currency)

        
    try:
        price_per_sqm = Selector(text=page.text).xpath(price_per_sqm_xpath).extract()[0]
        price_per_sqm = float(price_per_sqm.replace(',',''))
        price_per_sqm_list.append(price_per_sqm)
        #print(price_per_sqm)
    except:
        price_per_sqm = np.nan
        price_per_sqm_list.append(price_per_sqm)
    
    try:
        sqm_area = Selector(text=page.text).xpath(sqm_area_xpath).extract()[0]
        sqm_area = int(sqm_area)
        sqm_area_list.append(sqm_area)
        #print(sqm_area)
    except:
        sqm_area = np.nan
        sqm_area_list.append(sqm_area)
        
    try:
        reference_no = Selector(text=page.text).xpath(reference_no_xpath).extract()[0]
        reference_no_list.append(reference_no)
        #print(reference_no)
    except:
        reference_no = np.nan
        reference_no_list.append(reference_no)

    try:
        house_summary = Selector(text=page.text).xpath(house_summary_xpath).extract()[0]
        house_summary_list.append(house_summary)
        #print(house_summary)
    except:
        house_summary = np.nan
        house_summary_list.append(house_summary)
        
    try:
        region_name = Selector(text=page.text).xpath(region_name_xpath).extract()[0]
        region_name_list.append(region_name)
        #print(region_name)

        departmant_name = Selector(text=page.text).xpath(departmant_name_xpath).extract()[0]
        #print(departmant_name)
        departmant_name_list.append(departmant_name)

        city_name = Selector(text=page.text).xpath(city_name_xpath).extract()[0]
        #print(city_name)
        city_name_list.append(city_name)
    except:
        region_name = np.nan
        region_name_list.append(region_name)
        
        departmant_name = np.nan
        departmant_name_list.append(departmant_name)
        
        city_name = np.nan
        city_name_list.append(city_name)

    try:
        coor = re.search(r'data-address="(.*?)"', page.text).group(1)
        latitude =coor.split(',')[0]
        longitude = coor.split(',')[1]
        latitude_list.append(latitude)
        longitude_list.append(longitude)
    except:
        latitude = np.nan
        longitude = np.nan
        latitude_list.append(latitude)
        longitude_list.append(longitude)
    
    house_url_list.append(house_url)
    
    try:
        house_properties = Selector(text=page.text).xpath(house_properties_xpath).extract()[0]
        house_properties_list.append(house_properties)
    except:
        house_properties = np.nan
        house_properties_list.append(house_properties)
    
    try:
        try:
            sqm = re.search('\d+(?=\s(m))', house_properties).group(0)
            sqm_list.append(sqm_area)
        except:
            sqm = np.nan
            sqm_list.append(sqm_area)
        try:
            bedrooms = re.search('\d+(?=\s(bedrooms))', house_properties).group(0)
            bedrooms_list.append(bedrooms)
        except:
            bedrooms = np.nan
            bedrooms_list.append(bedrooms)
        try:
            shower_rooms = re.search('\d+(?=\s(shower))', house_properties).group(0)
            shower_rooms_list.append(shower_rooms)
        except:
            shower_rooms = np.nan
            shower_rooms_list.append(shower_rooms)
        try:
            bathrooms = re.search('\d+(?=\s(bathrooms))', house_properties).group(0)
            bathrooms_list.append(bathrooms)
        except:
            bathrooms = np.nan
            bathrooms_list.append(bathrooms)
        try:
            rooms = re.search('\d+(?=\s(rooms))', house_properties).group(0)
            rooms_list.append(rooms)
        except:
            rooms = np.nan
            rooms_list.append(rooms)
        try:
            ges = house_properties.split('GES')[1].split(')')[0].replace('(','')
            if "N/C" in ges:
                ges = np.nan
            ges_list.append(ges)
        except:
            ges = np.nan
            ges_list.append(ges)
        try:
            dpe = house_properties.split('DPE')[1].split(')')[0].replace('(','')
            if "N/C" in dpe or len(dpe)>12:
                dpe = np.nan
            dpe_list.append(dpe)                
        except:
            dpe = np.nan
            dpe_list.append(dpe)
    except Exception as e:
        print("Error: ",e)
    
    # Error popularity, fix it 
    try:
        popularity = Selector(text=page.text).xpath(popularity_xpath).extract()[0]
        #print(popularity)
        view_number = popularity.split('Seen ')[1].split(' ')[0]
        view_number_list.append(view_number)
        publish_date = popularity.split('the ')[1]
        publish_date_list.append(publish_date)
    except Exception as e:
        #print(popularity)
        view_number = np.nan
        view_number_list.append(view_number)
        publish_date = np.nan
        publish_date_list.append(publish_date)
        #print('something wrong in publish',e)
    
    #time.sleep(0.2)
    
    counter+=1
    if counter%250==0:
        print("250 house done")
    

end = time.time()
print(end - start," seconds execution time")

there is no url like: https://www.green-acres.fr/en/properties/28862a-060078459.htm
there is no url like: https://www.green-acres.fr/en/properties/31719a-1819441.htm
there is no url like: https://www.green-acres.fr/en/properties/13050a-340937762786.htm
there is no url like: https://www.green-acres.fr/en/properties/12848a-gre100036.htm
there is no url like: https://www.green-acres.fr/en/properties/54406a-7706-13-1807501.htm
there is no url like: https://www.green-acres.fr/en/properties/54406a-7706-13-2411427.htm
there is no url like: https://www.green-acres.fr/en/properties/44779a-01445.htm
there is no url like: https://www.green-acres.fr/en/properties/44708a-12388729.htm
there is no url like: https://www.green-acres.fr/en/properties/25282a-dg-9336.htm
there is no url like: https://www.green-acres.fr/en/properties/53673a-cr385.htm
250 house done
250 house done
250 house done
250 house done
250 house done
250 house done
250 house done
250 house done
250 house done
250 house done
250 hous

In [13]:
data_set = {'title':title_list,
            'price':price_value_list, 
            'price_currency':price_currency_list, 

            'price_per_sqm':price_per_sqm_list, 
            'sqm_area':sqm_area_list ,

            'reference_no':reference_no_list ,
            'house_summary':house_summary_list ,

            'region_name':region_name_list ,
            'departmant_name':departmant_name_list ,
            'city_name':city_name_list ,

            'latitude':latitude_list ,
            'longitude':longitude_list ,

            'house_url':house_url_list ,

            #'house_properties':house_properties_list ,
            #'view_number':view_number_list ,
            #'publish_date':publish_date_list,
            'sqm':sqm_list,
            'bedrooms':bedrooms_list,
            'rooms':rooms_list,
            'bathrooms':bathrooms_list,
            'shower room':shower_rooms_list,
            'ges':ges_list,
            'dpe':dpe_list}

In [25]:
df = pd.DataFrame(data=data_set,index=None)

In [26]:
df.tail()

,bathrooms,bedrooms,city_name,departmant_name,dpe,ges,house_summary,house_url,latitude,longitude,price,price_currency,price_per_sqm,reference_no,region_name,rooms,shower room,sqm,sqm_area,title
5347,3,3,Cannes,Alpes-Maritimes,NaN,NaN,"The prestigious residence ""First Croisette"" id...",https://www.green-acres.fr/en/properties/53807...,43.55087,7.01059,5500000.00000,€,36184.21000,Reference: 53807a-v5136ca,Provence-Alpes-Côte-d'Azur,4,NaN,152.00000,152.00000,Unique - First Croisette - High Standing Apart...
5348,NaN,2,Antibes,Alpes-Maritimes,F 344,F 80,2 bedroom apartment with a sea view and a larg...,https://www.green-acres.fr/en/properties/36814...,43.58072,7.12087,295000.00000,€,3782.05000,Reference: 36814a-2160424,Provence-Alpes-Côte-d'Azur,3,NaN,78.00000,78.00000,Sea View - 200 m beach - Antibes
5349,NaN,NaN,Roquebrune-Cap-Martin,Alpes-Maritimes,D 214,E 50,"In the city center and quiet, near beaches, re...",https://www.green-acres.fr/en/properties/51640...,43.76545,7.4588,240000.00000,€,nan,Professional property listing,Provence-Alpes-Côte-d'Azur,2,1,nan,nan,Sale studio furnished in 2 rooms Roquebrune Ca...
5350,NaN,3,Cannes,Alpes-Maritimes,NaN,NaN,"In luxury residence, splendid 3 bedroom sea vi...",https://www.green-acres.fr/en/properties/44708...,43.55087,7.01059,2100000.00000,€,nan,Agency: Home Hunts,Provence-Alpes-Côte-d'Azur,NaN,NaN,3.00000,3.00000,"In luxury residence, splendid 3 bedroom sea vi..."
5351,NaN,2,Contes,Alpes-Maritimes,NaN,NaN,Sandrine Zweifel vous propose A Sclos de Conte...,https://www.green-acres.fr/en/properties/7671a...,43.81198,7.31482,190000.00000,€,2835.82000,Reference: 7671a-27178,Provence-Alpes-Côte-d'Azur,3,NaN,67.00000,67.00000,"Urgent ! A Sclos de Contes, appartement 3 pièc..."


In [47]:
df.to_excel("./house_information_details.xlsx",index=False)

In [48]:
df_test = pd.read_excel("./house_information_details.xlsx")

In [50]:
df_test.head(3)

,bathrooms,bedrooms,city_name,departmant_name,dpe,ges,house_summary,house_url,latitude,longitude,price,price_currency,price_per_sqm,reference_no,region_name,rooms,shower room,sqm,sqm_area,title
0,nan,4.00000,Mougins,Alpes-Maritimes,NaN,NaN,Rare. A Few Minutes From The Village Of Mougin...,https://www.green-acres.fr/en/properties/4732a...,43.59870,7.00430,1950000.00000,€,nan,Professional property listing,Provence-Alpes-Côte-d'Azur,4.00000,nan,220.00000,220.00000,Few minutes away from Mougins
1,nan,3.00000,Menton,Alpes-Maritimes,F 423,D 26,Full textual description available only on:&nb...,https://www.green-acres.fr/en/properties/37892...,43.77885,7.50567,937000.00000,€,7207.69000,Reference: 37892a-fr385369,Provence-Alpes-Côte-d'Azur,5.00000,3.00000,130.00000,130.00000,House: 130 m²
2,nan,nan,Nice,Alpes-Maritimes,NaN,NaN,"Superb apartment 2 P duplex renovated, luxury ...",https://www.green-acres.fr/en/properties/55720...,43.70341,7.26620,179000.00000,€,7160.00000,Reference: 55720a-1632590,Provence-Alpes-Côte-d'Azur,2.00000,nan,25.00000,25.00000,Duplex apartment 2P


In [52]:
max(df_test['price'])

69000000.0